In [ ]:
import subprocess
import time
import os

script_name = "subprocess_run.py"
script_path = os.path.join(os.getcwd(), script_name)

# subprocess_run.py 실행
process = subprocess.Popen(
    f'python {script_path}',
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)

# 프로세스 완료 대기
process.wait()

print(f"Process finished with return code {process.returncode}")


In [1]:
import run_simulation
run_simulation.main(test=True)

2025-07-26 21:48:45,926 - INFO - ==========simulation29==========
PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
2025-07-26 21:48:45,927 - INFO - Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
2025-07-26 21:48:45,929 - INFO - PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
2025-07-26 21:48:45,930 - INFO - Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
2025-07-26 21:48:45,932 - INFO - Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_1e332609-f9c6-4e35-9428-483519037b85.log is enabled.
2025-07-26 21:48:45,933 - INFO - Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_1e332609-f9c6-4e35-9428-483519037b85.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
2025-07-26 21:48:45,934 - INFO - 

In [1]:
import sys
import os
import platform
import shutil

os_name = platform.system()
if os_name == "Windows":
    sys.path.insert(0, r"Y:/git/insulation_amp/pyaedt_library/src/") 
else :
    sys.path.insert(0, r"/gpfs/home1/r1jae262/jupyter/git/pyaedt_library/src/")

import pyaedt_module
from pyaedt_module.core import pyDesktop

import time
from datetime import datetime

import math
import copy

import pandas as pd

import csv
from filelock import FileLock
import traceback
import logging

from module.input_parameter import create_input_parameter, create_input_parameter_for_test, calculate_coil_parameter, calculate_coil_offset, set_design_variables
from module.modeling import (
    create_core_model, create_all_windings, create_cold_plate, create_air,
    assign_meshing, assign_excitations, create_face, create_mold
)
from module.report import (
    get_input_parameter, get_maxwell_magnetic_parameter,
    get_maxwell_calculator_parameter, get_convergence_report, get_icepak_calculator_parameter
)


def save_error_log(project_name, error_info):
    error_folder = "error"
    os.makedirs(error_folder, exist_ok=True)
    error_file = os.path.join(error_folder, f"{project_name}_error.txt")
    with open(error_file, "w", encoding="utf-8") as f:
        f.write(error_info)


class Simulation() :

    def __init__(self) :

        self.NUM_CORE = 4
        self.NUM_TASK = 1

        file_path = "simulation_num.txt"

        # 파일이 존재하지 않으면 생성
        if not os.path.exists(file_path):
            with open(file_path, "w", encoding="utf-8") as file:
                file.write("1")

        # 읽기/쓰기 모드로 파일 열기
        with open(file_path, "r+", encoding="utf-8") as file:

            # 파일에서 값 읽기
            content = int(file.read().strip())
            self.num = content
            self.PROJECT_NAME = f"simulation{content}"
            content += 1

            # 파일 포인터를 처음으로 되돌리고, 파일 내용 초기화 후 새 값 쓰기
            file.seek(0)
            file.truncate()
            file.write(str(content))

            # 파일은 with 블록 종료 시 자동으로 닫히며, 잠금도 해제됨

        print(f"==========simulation{content}==========")

        os_name = platform.system()
        if os_name == "Windows":
            GUI = False
        else :
            GUI = True

        self.desktop = pyDesktop(version=None, non_graphical=GUI)


    def create_design(self, name) :
        self.project = self.desktop.create_project()
        self.maxwell_design = self.project.create_design(name=name, solver="Maxwell3d", solution=None)
        return self.maxwell_design

    
    def create_input_parameter(self, param_list=None) :
        if self.test == True :
            input_parameter = create_input_parameter_for_test(self.maxwell_design, param_list)
        else :
            input_parameter = create_input_parameter(self.maxwell_design, param_list)
        print(f"input_parameter : {input_parameter}")
        print("input_parameter :", ",".join(str(float(v)) for v in input_parameter.values()))
        return input_parameter


    def set_variable(self, input_parameter) :
        # 1. Simulation 클래스 인스턴스에 속성으로 값을 설정합니다 (예: self.N1 = 10).
        for key, value in input_parameter.items():
            setattr(self.maxwell_design, key, value)
        
        self.input_df = pd.DataFrame([input_parameter])
        
        # 2. input_parameter.py의 함수를 호출하여 Ansys 디자인에 변수를 설정합니다.
        set_design_variables(self.maxwell_design, input_parameter)

    def set_maxwell_analysis(self) :
        self.maxwell_design.setup = self.maxwell_design.create_setup(name = "Setup1")
        self.maxwell_design.setup.properties["Max. Number of Passes"] = 3 # 10
        self.maxwell_design.setup.properties["Min. Number of Passes"] = 1
        self.maxwell_design.setup.properties["Percent Error"] = 2.5 # 2.5
        self.maxwell_design.setup.properties["Frequency Setup"] = f"{self.maxwell_design.frequency}kHz"

    def create_core(self):
        # self.maxwell_design.set_power_ferrite(cm=0.2435*1e-3, x=2.2, y=2.299)
        self.maxwell_design.set_power_ferrite(cm=2.315, x=1.4472, y=2.4769)
        self.maxwell_design.core = create_core_model(self.maxwell_design)


    def create_face(self, design_obj):
        create_face(design_obj)


    def create_windings(self):
        """Creates both primary and secondary windings."""
        self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3 = create_all_windings(self.maxwell_design)


    def create_mold(self):
        self.maxwell_design.mold = create_mold(self.maxwell_design)
        self.maxwell_design.modeler.subtract(
						blank_list = [self.maxwell_design.mold],
						tool_list = [self.maxwell_design.core, self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3],
						keep_originals = True
					)

    def create_cold_plate(self):
        """Creates the cold plates."""
        self.maxwell_design.cold_plate_top, self.maxwell_design.cold_plate_bottom = create_cold_plate(self.maxwell_design)

    def create_air(self):
        """Creates the air region."""
        self.maxwell_design.air_region = create_air(self.maxwell_design)

    def assign_meshing(self):
        """Assigns mesh operations."""
        self.maxwell_design.length_mesh, self.maxwell_design.skin_depth_mesh = assign_meshing(self.maxwell_design)

    def assign_excitations(self):
        """Identifies terminals and assigns all excitations."""
        self.maxwell_design.Tx_winding, self.maxwell_design.Rx_winding1, self.maxwell_design.Rx_winding2 = assign_excitations(self.maxwell_design, self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3)

    def analyze_maxwell(self, design):
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        os.makedirs(folder_path, exist_ok=True)  # 폴더가 없으면 생성
        file_path = os.path.join(folder_path, f"{self.PROJECT_NAME}.aedt")
        self.project.save_project(path=file_path)
        
        start_time = time.time()
        print(f"Maxwell analysis started...")
        design.analyze()
        end_time = time.time()
        print(f"Maxwell analysis finished. Duration: {end_time - start_time:.2f} seconds.")

    def get_simulation_results(self, design=None,input=True, step=1):
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        if input:
            input_df = get_input_parameter(self.maxwell_design)
            result_df = pd.concat([input_df], axis=1)
        else:
            result_df = self.results_df # Use existing df for second run

        
        if step == 1:
            design.magnetic_report, magnetic_df = get_maxwell_magnetic_parameter(design, dir=folder_path, mod="write", import_report=None, report_name="magnetic_report", file_name="magnetic_report1")
            design.calculator_report, calculator_df = get_maxwell_calculator_parameter(design, dir=folder_path, mod="write", import_report=None, report_name="calculator_report", file_name="calculator_report1")
        else:
            design.magnetic_report, magnetic_df = get_maxwell_magnetic_parameter(design, dir=folder_path, mod="read", import_report=self.maxwell_design.magnetic_report, report_name="magnetic_report", file_name="magnetic_report2")
            design.calculator_report, calculator_df = get_maxwell_calculator_parameter(design, dir=folder_path, mod="read", import_report=self.maxwell_design.calculator_report, report_name="calculator_report", file_name="calculator_report2")
        analysis_df = get_convergence_report(design)

        results_df = pd.concat([result_df, magnetic_df, calculator_df, analysis_df], axis=1)
        self.results_df = results_df

        return results_df

    def save_results_to_csv(self, results_df, filename="simulation_results.csv"):
        """Saves the DataFrame to a CSV file in a process-safe way."""
        lock_path = filename + ".lock"
        with FileLock(lock_path):
            file_exists = os.path.isfile(filename)
            results_df.to_csv(filename, mode='a', header=not file_exists, index=False)
        print(f"Results saved to {filename}")


    def second_simulation(self):

        time.sleep(5)

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oProject.CopyDesign(self.maxwell_design.name)
        oProject.Paste()

        self.maxwell_design2 = self.maxwell_design.get_active_design()

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = oProject.SetActiveDesign(self.maxwell_design2.design_name)

        # Map cold plate objects from Maxwell to Maxwell2
        self.maxwell_design2.cold_plate_top = self.maxwell_design2.model3d.find_object(self.maxwell_design.cold_plate_top)
        self.maxwell_design2.cold_plate_bottom = self.maxwell_design2.model3d.find_object(self.maxwell_design.cold_plate_bottom)
        self.maxwell_design2.winding1 = self.maxwell_design2.model3d.find_object(self.maxwell_design.winding1)
        self.maxwell_design2.winding2 = self.maxwell_design2.model3d.find_object(self.maxwell_design.winding2)
        self.maxwell_design2.winding3 = self.maxwell_design2.model3d.find_object(self.maxwell_design.winding3)
        self.maxwell_design2.core = self.maxwell_design2.model3d.find_object(self.maxwell_design.core)  
        self.maxwell_design2.leg_left = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_left)
        self.maxwell_design2.leg_right = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_right)
        self.maxwell_design2.leg_center = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_center)
        self.maxwell_design2.leg_top_left = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_top_left)
        self.maxwell_design2.leg_top_right = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_top_right)
        self.maxwell_design2.leg_bottom_left = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_bottom_left)
        self.maxwell_design2.leg_bottom_right = self.maxwell_design2.model3d.find_object(self.maxwell_design.leg_bottom_right)

        self.maxwell_design2.delete_mesh(self.maxwell_design.skin_depth_mesh)
        
        V = 750
        Im = V/2/3.141592/self.maxwell_design.frequency/1e3/(float(self.results_df["Lmt1"])*1e-6)

        # re excitation
        excitation_list=[self.maxwell_design.Tx_winding.name, self.maxwell_design.Rx_winding1.name, self.maxwell_design.Rx_winding2.name]
        [self.maxwell_design2.Tx_winding, self.maxwell_design2.Rx_winding1, self.maxwell_design2.Rx_winding2] = self.maxwell_design2.get_excitation(excitation_name=excitation_list)

        self.maxwell_design2.Tx_winding["Current"] = f'{Im} * sqrt(2)A'
        self.maxwell_design2.Rx_winding1["Current"] = '0 * sqrt(2)A'
        self.maxwell_design2.Rx_winding2["Current"] = '0 * sqrt(2)A'

        self.maxwell_design2.setup = self.maxwell_design2.get_setup(name="Setup1")
        self.maxwell_design2.setup.properties["Max. Number of Passes"] = 5 # 10
        self.maxwell_design2.setup.properties["Percent Error"] = 3 # 2.5


        self.analyze_maxwell(self.maxwell_design2)
        self.get_simulation_results(design=self.maxwell_design2, input=False, step=2)

    def create_icepak(self):

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = oProject.SetActiveDesign(self.maxwell_design.design_name)
        oDesign.CreateEMLossTarget("Icepak", "Setup1 : LastAdaptive", 
            [
                "NAME:DesignSetup",
                "Sim Type:="		, "Forced"
            ])
        self.icepak_design = self.maxwell_design.get_active_design()

        return self.icepak_design

    def setup_icepak_analysis(self):
        """Sets up the Icepak analysis, including loss mapping and boundary conditions."""

        self.icepak_design.set_ambient_temp(temp=30)

        # Recreate non-model sheet objects in Icepak for result evaluation
        create_face(self.icepak_design)


        # Delete default EM loss boundary that is automatically created        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = self.icepak_design.odesign
        oModule = oDesign.GetModule("BoundarySetup")
        oModule.DeleteBoundaries(["EMLoss1"])

        # Assign EM Loss from Maxwell analysis to Icepak thermal simulation
        self.icepak_design.assign_EM_loss(name="Coreloss", objects=[self.maxwell_design.core], design=self.maxwell_design2, frequency=self.maxwell_design.frequency*1e+3, loss_mul=1)
        self.icepak_design.assign_EM_loss(name="Windingloss", objects=[self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3], 
                                          design=self.maxwell_design, frequency=self.maxwell_design.frequency*1e+3, loss_mul=self.maxwell_design.winding_thermal_ratio)
        # self.icepak_design.assign_EM_loss(name="Windingloss", objects=[self.maxwell_design.winding1, self.maxwell_design.winding2], design=self.maxwell_design, frequency=self.freq, loss_mul=1)

        # boundary setup
        self.icepak_design.boundaries[1].properties["X Velocity"] = f"{self.maxwell_design.wind_speed}m_per_sec"

        # Configure Icepak analysis setup
        self.icepak_design.icepak_cetup = self.icepak_design.get_setup(name="Setup1")
        self.icepak_design.icepak_cetup.props["Flow Regime"] = "Turbulent"
        self.icepak_design.icepak_cetup.props["Include Gravity"] = True
        self.icepak_design.icepak_cetup.props["Solution Initialization - Z Velocity"] = "0.1m_per_sec"

        # Map cold plate objects from Maxwell to Icepak
        self.icepak_design.cold_plate_top = self.icepak_design.model3d.find_object(self.maxwell_design.cold_plate_top)
        self.icepak_design.cold_plate_bottom = self.icepak_design.model3d.find_object(self.maxwell_design.cold_plate_bottom)
        self.icepak_design.winding1 = self.icepak_design.model3d.find_object(self.maxwell_design.winding1)
        self.icepak_design.winding2 = self.icepak_design.model3d.find_object(self.maxwell_design.winding2)
        self.icepak_design.winding3 = self.icepak_design.model3d.find_object(self.maxwell_design.winding3)
        self.icepak_design.core = self.icepak_design.model3d.find_object(self.maxwell_design.core)  
        self.icepak_design.leg_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_left)
        self.icepak_design.leg_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_right)
        self.icepak_design.leg_center = self.icepak_design.model3d.find_object(self.maxwell_design.leg_center)
        self.icepak_design.leg_top_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_top_left)
        self.icepak_design.leg_top_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_top_right)
        self.icepak_design.leg_bottom_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_bottom_left)
        self.icepak_design.leg_bottom_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_bottom_right)

        

        # Assign fixed temperature boundary condition to the cold plates
        self.icepak_design.assign_icepak_source(
            assignment=[self.icepak_design.cold_plate_top, self.icepak_design.cold_plate_bottom], 
            thermal_condition="Fixed Temperature", 
            assignment_value="AmbientTemp", 
            boundary_name="cold_plate"
        )

    def analyze_icepak(self):
        """Runs the Icepak analysis."""
        start_time = time.time()
        print(f"Icepak analysis started...")
        self.icepak_design.analyze()
        end_time = time.time()
        print(f"Icepak analysis finished. Duration: {end_time - start_time:.2f} seconds.")

    def get_icepak_results(self):
        """Retrieves and processes results from the Icepak simulation."""
        

        # Define the temperature parameters to be extracted
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        
        # Get results using the calculator
        self.icepak_design.calculator_report, icepak_results_df = get_icepak_calculator_parameter(
            self.icepak_design, dir=folder_path, mod="write", report_name="thermal_report", file_name="thermal_report1"
        )
        
        # Concatenate Icepak results with the main results DataFrame
        self.results_df = pd.concat([self.results_df, icepak_results_df], axis=1)


    def close_project(self):
        self.maxwell_design.cleanup_solution()
        self.icepak_design.cleanup_solution()

        self.maxwell_design.close_project()

        self.desktop.release_desktop(close_projects=True, close_on_exit=True)

    def delete_project_folder(self):
        time.sleep(10)
        try:
            project_folder = os.path.join(os.getcwd(), "simulation", self.PROJECT_NAME)
            if os.path.isdir(project_folder):
                shutil.rmtree(project_folder)
                print(f"Successfully deleted project folder: {project_folder}")
        except Exception as e:
            print(f"Error deleting project folder {project_folder}: {e}", file=sys.stderr)



test = True
simulation_runner = Simulation()

if test == True :
    simulation_runner.test = True
else :
    simulation_runner.test = False

# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
input_parameters = simulation_runner.create_input_parameter()

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()

simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell(simulation_runner.maxwell_design)

# 9. 결과 리포팅
simulation_runner.get_simulation_results(design=simulation_runner.maxwell_design, input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()

# 15. 결과 저장
simulation_runner.save_results_to_csv(simulation_runner.results_df)



==========simulation11==========
PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_ee279672-147a-4732-8a75-b6156299ca7a.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 50000.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 50000 after 6.900363206863403 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v2

In [5]:
def get_free_ports(num_ports):
    """
    사용 가능한 GRPC 포트를 찾아서 반환합니다.
    
    Args:
        num_ports (int): 필요한 포트 개수
        
    Returns:
        list: 사용 가능한 포트 번호 리스트
    """
    import socket
    
    def is_port_available(port):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            sock.bind(('localhost', port))
            sock.close()
            return True
        except:
            return False
            
    available_ports = []
    port = 50000  # GRPC 기본 포트 범위 시작
    
    while len(available_ports) < num_ports:
        if is_port_available(port):
            available_ports.append(port)
        port += 1
        if port > 60000:  # 최대 포트 번호 제한
            break
            
    return available_ports

# 사용 예시: 6개의 사용 가능한 포트 찾기
free_ports = get_free_ports(6)
print(f"사용 가능한 GRPC 포트: {free_ports[0]}")


사용 가능한 GRPC 포트: 50000


In [1]:
import sys
import os
import platform
import shutil

os_name = platform.system()
if os_name == "Windows":
    sys.path.insert(0, r"Y:/git/insulation_amp/pyaedt_library/src/") 
else :
    sys.path.insert(0, r"/gpfs/home1/r1jae262/jupyter/git/pyaedt_library/src/")

import pyaedt_module
from pyaedt_module.core import pyDesktop

import time
from datetime import datetime

import math
import copy

import pandas as pd

import csv
from filelock import FileLock
import traceback
import logging

from module.input_parameter import create_input_parameter, calculate_coil_parameter, calculate_coil_offset, set_design_variables
from module.modeling import (
    create_core_model, create_all_windings, create_cold_plate, create_air,
    assign_meshing, assign_excitations, create_face, create_mold
)
from module.report import (
    get_input_parameter, get_maxwell_magnetic_parameter,
    get_maxwell_calculator_parameter, get_convergence_report, get_icepak_calculator_parameter
)


def save_error_log(project_name, error_info):
    error_folder = "error"
    os.makedirs(error_folder, exist_ok=True)
    error_file = os.path.join(error_folder, f"{project_name}_error.txt")
    with open(error_file, "w", encoding="utf-8") as f:
        f.write(error_info)


class Simulation() :

    def __init__(self) :

        self.NUM_CORE = 4
        self.NUM_TASK = 1

        file_path = "simulation_num.txt"

        # 파일이 존재하지 않으면 생성
        if not os.path.exists(file_path):
            with open(file_path, "w", encoding="utf-8") as file:
                file.write("1")

        # 읽기/쓰기 모드로 파일 열기
        with open(file_path, "r+", encoding="utf-8") as file:

            # 파일에서 값 읽기
            content = int(file.read().strip())
            self.num = content
            self.PROJECT_NAME = f"simulation{content}"
            content += 1

            # 파일 포인터를 처음으로 되돌리고, 파일 내용 초기화 후 새 값 쓰기
            file.seek(0)
            file.truncate()
            file.write(str(content))

            # 파일은 with 블록 종료 시 자동으로 닫히며, 잠금도 해제됨

        print(f"==========simulation{content}==========")

        os_name = platform.system()
        if os_name == "Windows":
            GUI = False
        else :
            GUI = True

        self.desktop = pyDesktop(version=None, non_graphical=GUI)


    def create_design(self, name) :
        self.project = self.desktop.create_project()
        self.maxwell_design = self.project.create_design(name=name, solver="Maxwell3d", solution=None)
        return self.maxwell_design

    
    def create_input_parameter(self, param_list=None) :
        input_parameter = create_input_parameter(self.maxwell_design, param_list)
        print(f"input_parameter : {input_parameter}")
        print("input_parameter :", ",".join(str(float(v)) for v in input_parameter.values()))
        return input_parameter


    def set_variable(self, input_parameter) :
        # 1. Simulation 클래스 인스턴스에 속성으로 값을 설정합니다 (예: self.N1 = 10).
        for key, value in input_parameter.items():
            setattr(self.maxwell_design, key, value)
        
        self.input_df = pd.DataFrame([input_parameter])
        
        # 2. input_parameter.py의 함수를 호출하여 Ansys 디자인에 변수를 설정합니다.
        set_design_variables(self.maxwell_design, input_parameter)

    def set_maxwell_analysis(self) :
        self.maxwell_design.setup = self.maxwell_design.create_setup(name = "Setup1")
        self.maxwell_design.setup.properties["Max. Number of Passes"] = 3 # 10
        self.maxwell_design.setup.properties["Min. Number of Passes"] = 1
        self.maxwell_design.setup.properties["Percent Error"] = 5 # 2.5
        self.maxwell_design.setup.properties["Frequency Setup"] = f"{self.maxwell_design.frequency}kHz"

    def create_core(self):
        # self.maxwell_design.set_power_ferrite(cm=0.2435*1e-3, x=2.2, y=2.299)
        self.maxwell_design.set_power_ferrite(cm=2.315, x=1.4472, y=2.4769)
        self.maxwell_design.core = create_core_model(self.maxwell_design)


    def create_face(self, design_obj):
        create_face(design_obj)


    def create_windings(self):
        """Creates both primary and secondary windings."""
        self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3 = create_all_windings(self.maxwell_design)


    def create_mold(self):
        self.maxwell_design.mold = create_mold(self.maxwell_design)
        self.maxwell_design.modeler.subtract(
						blank_list = [self.maxwell_design.mold],
						tool_list = [self.maxwell_design.core, self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3],
						keep_originals = True
					)

    def create_cold_plate(self):
        """Creates the cold plates."""
        self.maxwell_design.cold_plate_top, self.maxwell_design.cold_plate_bottom = create_cold_plate(self.maxwell_design)

    def create_air(self):
        """Creates the air region."""
        self.maxwell_design.air_region = create_air(self.maxwell_design)

    def assign_meshing(self):
        """Assigns mesh operations."""
        self.maxwell_design.length_mesh, self.maxwell_design.skin_depth_mesh = assign_meshing(self.maxwell_design)

    def assign_excitations(self):
        """Identifies terminals and assigns all excitations."""
        self.maxwell_design.Tx_winding, self.maxwell_design.Rx_winding1, self.maxwell_design.Rx_winding2 = assign_excitations(self.maxwell_design, self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3)

    def analyze_maxwell(self, design):
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        os.makedirs(folder_path, exist_ok=True)  # 폴더가 없으면 생성
        file_path = os.path.join(folder_path, f"{self.PROJECT_NAME}.aedt")
        self.project.save_project(path=file_path)
        
        start_time = time.time()
        print(f"Maxwell analysis started...")
        design.analyze()
        end_time = time.time()
        print(f"Maxwell analysis finished. Duration: {end_time - start_time:.2f} seconds.")

    def get_simulation_results(self, input=True, step=1):
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        if input:
            input_df = get_input_parameter(self.maxwell_design)
            result_df = pd.concat([input_df], axis=1)
        else:
            result_df = self.results_df # Use existing df for second run

        
        if step == 1:
            self.maxwell_design.magnetic_report, magnetic_df = get_maxwell_magnetic_parameter(self.maxwell_design, dir=folder_path, mod="write", import_report=None)
            self.maxwell_design.calculator_report, calculator_df = get_maxwell_calculator_parameter(self.maxwell_design, dir=folder_path, mod="write", import_report=None)
        else:
            self.maxwell_design.magnetic_report, magnetic_df = get_maxwell_magnetic_parameter(self.maxwell_design, dir=folder_path, mod="read", import_report=self.maxwell_design.magnetic_report)
            self.maxwell_design.calculator_report, calculator_df = get_maxwell_calculator_parameter(self.maxwell_design, dir=folder_path, mod="read", import_report=self.maxwell_design.calculator_report)
        analysis_df = get_convergence_report(self.maxwell_design)

        results_df = pd.concat([result_df, magnetic_df, calculator_df, analysis_df], axis=1)
        self.results_df = results_df

        return results_df

    def save_results_to_csv(self, results_df, filename="simulation_results.csv"):
        """Saves the DataFrame to a CSV file in a process-safe way."""
        lock_path = filename + ".lock"
        with FileLock(lock_path):
            file_exists = os.path.isfile(filename)
            results_df.to_csv(filename, mode='a', header=not file_exists, index=False)
        print(f"Results saved to {filename}")


    def second_simulation(self):

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oProject.CopyDesign(self.maxwell_design.name)
        oProject.Paste()

        self.maxwell_design2 = self.maxwell_design.get_active_design()

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = oProject.SetActiveDesign(self.maxwell_design2.design_name)

        self.maxwell_design2.delete_mesh(self.maxwell_design.skin_depth_mesh)
        
        V = 750
        Im = V/2/3.141592/self.maxwell_design.frequency/1e3/(float(self.results_df["Lmt1"])*1e-6)

        # re excitation
        excitation_list=[self.maxwell_design.Tx_winding.name, self.maxwell_design.Rx_winding1.name, self.maxwell_design.Rx_winding2.name]
        [self.maxwell_design2.Tx_winding, self.maxwell_design2.Rx_winding1, self.maxwell_design2.Rx_winding2] = self.maxwell_design2.get_excitation(excitation_name=excitation_list)

        self.maxwell_design2.Tx_winding["Current"] = f'{Im} * sqrt(2)A'
        self.maxwell_design2.Rx_winding1["Current"] = '0 * sqrt(2)A'
        self.maxwell_design2.Rx_winding2["Current"] = '0 * sqrt(2)A'

        self.maxwell_design2.setup = self.maxwell_design2.get_setup(name="Setup1")
        self.maxwell_design2.setup.properties["Max. Number of Passes"] = 10 # 10
        self.maxwell_design2.setup.properties["Percent Error"] = 1 # 2.5


        self.analyze_maxwell(self.maxwell_design2)
        self.get_simulation_results(input=False, step=2)

    def create_icepak(self):

        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = oProject.SetActiveDesign(self.maxwell_design.design_name)
        oDesign.CreateEMLossTarget("Icepak", "Setup1 : LastAdaptive", 
            [
                "NAME:DesignSetup",
                "Sim Type:="		, "Forced"
            ])
        self.icepak_design = self.maxwell_design.get_active_design()

        return self.icepak_design

    def setup_icepak_analysis(self):
        """Sets up the Icepak analysis, including loss mapping and boundary conditions."""

        self.icepak_design.set_ambient_temp(temp=30)

        # Recreate non-model sheet objects in Icepak for result evaluation
        create_face(self.icepak_design)


        # Delete default EM loss boundary that is automatically created        oProject = self.desktop.odesktop.SetActiveProject(self.project.name)
        oDesign = self.icepak_design.odesign
        oModule = oDesign.GetModule("BoundarySetup")
        oModule.DeleteBoundaries(["EMLoss1"])

        # Assign EM Loss from Maxwell analysis to Icepak thermal simulation
        self.icepak_design.assign_EM_loss(name="Coreloss", objects=[self.maxwell_design.core], design=self.maxwell_design2, frequency=self.maxwell_design.frequency*1e+3, loss_mul=1)
        self.icepak_design.assign_EM_loss(name="Windingloss", objects=[self.maxwell_design.winding1, self.maxwell_design.winding2, self.maxwell_design.winding3], 
                                          design=self.maxwell_design, frequency=self.maxwell_design.frequency*1e+3, loss_mul=self.maxwell_design.winding_thermal_ratio)
        # self.icepak_design.assign_EM_loss(name="Windingloss", objects=[self.maxwell_design.winding1, self.maxwell_design.winding2], design=self.maxwell_design, frequency=self.freq, loss_mul=1)

        # boundary setup
        self.icepak_design.boundaries[1].properties["X Velocity"] = f"{self.maxwell_design.wind_speed}m_per_sec"

        # Configure Icepak analysis setup
        self.icepak_design.icepak_cetup = self.icepak_design.get_setup(name="Setup1")
        self.icepak_design.icepak_cetup.props["Flow Regime"] = "Turbulent"
        self.icepak_design.icepak_cetup.props["Include Gravity"] = True
        self.icepak_design.icepak_cetup.props["Solution Initialization - Z Velocity"] = "0.1m_per_sec"

        # Map cold plate objects from Maxwell to Icepak
        self.icepak_design.cold_plate_top = self.icepak_design.model3d.find_object(self.maxwell_design.cold_plate_top)
        self.icepak_design.cold_plate_bottom = self.icepak_design.model3d.find_object(self.maxwell_design.cold_plate_bottom)
        self.icepak_design.winding1 = self.icepak_design.model3d.find_object(self.maxwell_design.winding1)
        self.icepak_design.winding2 = self.icepak_design.model3d.find_object(self.maxwell_design.winding2)
        self.icepak_design.winding3 = self.icepak_design.model3d.find_object(self.maxwell_design.winding3)
        self.icepak_design.core = self.icepak_design.model3d.find_object(self.maxwell_design.core)  
        self.icepak_design.leg_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_left)
        self.icepak_design.leg_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_right)
        self.icepak_design.leg_center = self.icepak_design.model3d.find_object(self.maxwell_design.leg_center)
        self.icepak_design.leg_top_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_top_left)
        self.icepak_design.leg_top_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_top_right)
        self.icepak_design.leg_bottom_left = self.icepak_design.model3d.find_object(self.maxwell_design.leg_bottom_left)
        self.icepak_design.leg_bottom_right = self.icepak_design.model3d.find_object(self.maxwell_design.leg_bottom_right)

        # Assign fixed temperature boundary condition to the cold plates
        self.icepak_design.assign_icepak_source(
            assignment=[self.icepak_design.cold_plate_top, self.icepak_design.cold_plate_bottom], 
            thermal_condition="Fixed Temperature", 
            assignment_value="AmbientTemp", 
            boundary_name="cold_plate"
        )

    def analyze_icepak(self):
        """Runs the Icepak analysis."""
        start_time = time.time()
        print(f"Icepak analysis started...")
        self.icepak_design.analyze()
        end_time = time.time()
        print(f"Icepak analysis finished. Duration: {end_time - start_time:.2f} seconds.")

    def get_icepak_results(self):
        """Retrieves and processes results from the Icepak simulation."""
        

        # Define the temperature parameters to be extracted
        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        
        # Get results using the calculator
        self.icepak_design.calculator_report, icepak_results_df = get_icepak_calculator_parameter(
            self.icepak_design, dir=folder_path
        )
        
        # Concatenate Icepak results with the main results DataFrame
        self.results_df = pd.concat([self.results_df, icepak_results_df], axis=1)


    def close_project(self):
        self.maxwell_design.cleanup_solution()
        self.icepak_design.cleanup_solution()

        self.maxwell_design.close_project()

        self.desktop.release_desktop(close_projects=True, close_on_exit=True)

    def delete_project_folder(self):
        time.sleep(10)
        try:
            project_folder = os.path.join(os.getcwd(), "simulation", self.PROJECT_NAME)
            if os.path.isdir(project_folder):
                shutil.rmtree(project_folder)
                print(f"Successfully deleted project folder: {project_folder}")
        except Exception as e:
            print(f"Error deleting project folder {project_folder}: {e}", file=sys.stderr)

In [2]:
simulation_runner = Simulation()
# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
input_parameters = simulation_runner.create_input_parameter()

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()

simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell(simulation_runner.maxwell_design)

# 9. 결과 리포팅
simulation_runner.get_simulation_results(input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()

==========simulation63==========
PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_4b4275c4-91b4-4cae-926c-0e26ea477881.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 57688.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 57688 after 5.647041320800781 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v2

In [ ]:
test = False

for i in range(5000):
    try:
        simulation_runner = Simulation()

        if test == True :
            simulation_runner.test = True
        else :
            simulation_runner.test = False

        # 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
        simulation_runner.create_design("SST_MFT")

        # 3. 이제 입력 매개변수를 생성할 수 있습니다.
        input_parameters = simulation_runner.create_input_parameter()

        # 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
        simulation_runner.set_variable(input_parameters)

        # 5. 해석 설정 및 실행
        simulation_runner.set_maxwell_analysis()

        # 6. 모델을 생성합니다.
        simulation_runner.create_core()

        simulation_runner.create_face(simulation_runner.maxwell_design)
        simulation_runner.create_windings()
        simulation_runner.create_mold()
        simulation_runner.create_cold_plate()
        simulation_runner.create_air()

        # 7. 메쉬 및 경계조건 설정
        simulation_runner.assign_meshing()
        simulation_runner.assign_excitations()

        # 8. 해석 설정 및 실행
        simulation_runner.analyze_maxwell(simulation_runner.maxwell_design)

        # 9. 결과 리포팅
        simulation_runner.get_simulation_results(design=simulation_runner.maxwell_design, input=True)

        # 10. 두 번째 해석 실행
        simulation_runner.second_simulation()

        # 11. Icepak 디자인 생성
        simulation_runner.create_icepak()       

        # 12. Icepak 해석 설정
        simulation_runner.setup_icepak_analysis()

        # 13. Icepak 해석 실행
        simulation_runner.analyze_icepak()

        # 14. Icepak 결과 리포팅
        simulation_runner.get_icepak_results()

        # 15. 결과 저장
        simulation_runner.save_results_to_csv(simulation_runner.results_df)

        if test == False :

            simulation_runner.close_project()
            simulation_runner.delete_project_folder()

    except Exception as e:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        
        err_info = f"error : {simulation_runner.PROJECT_NAME}\n"
        try:
            err_info += f"input : {simulation_runner.input_df.to_string()}\n"
        except AttributeError:
            err_info += "input : Not available\n"
        
        err_info += f"{str(e)}\n"
        err_info += traceback.format_exc()

        print(err_info, file=sys.stderr)
        sys.stderr.flush()
        
        logging.error(err_info, exc_info=True)
        
        save_error_log(simulation_runner.PROJECT_NAME, err_info)
        
        print(f"{simulation_runner.PROJECT_NAME} : {i} simulation Failed")
        
        simulation_runner.desktop.release_desktop(close_projects=True, close_on_exit=True)

        time.sleep(10)

==========simulation3==========
PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].


INFO:Global:Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.15.0.


INFO:Global:PyAEDT version 0.15.0.


PyAEDT INFO: Initializing new Desktop session.


...

PyAEDT INFO: PyAEDT version 0.15.0.


INFO:Global:PyAEDT version 0.15.0.


PyAEDT INFO: Returning found Desktop session with PID 35004!


INFO:Global:Returning found Desktop session with PID 35004!


PyAEDT INFO: No project is defined. Project Project_024 exists and has been read.


In [ ]:
simulation_runner.icepak_design.boundaries[1].properties["X Velocity"] = "10m_per_sec"

In [ ]:
simulation_runner.maxwell_design

In [ ]:
excitation_name=[simulation_runner.maxwell_design.Tx_winding.name, simulation_runner.maxwell_design.Rx_winding1.name, simulation_runner.maxwell_design.Rx_winding2.name]

simulation_runner.maxwell_design2.get_excitation(excitation_name)

In [ ]:
simulation_runner.maxwell_design.skin_depth_mesh.properties

simulation_runner.maxwell_design.excitation_objects

for name, bound_obj in simulation_runner.maxwell_design.excitation_objects.items():
    print(f"{name}: {bound_obj}")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
simulation_runner.results_df

In [ ]:
simulation_runner.maxwell_design.Tx_winding["Current"]

In [ ]:
import run_simulation

# 1. Simulation 클래스의 인스턴스를 생성합니다.
simulation_runner = run_simulation.Simulation()

# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
input_parameters2 = simulation_runner.create_input_parameter()
import numpy as np

# Define the keys in the correct order
keys = [
    "N1", "N2", "N1_layer", "N2_layer", "per", "w1", "l1", "l2", "l2_gap", "h1",
    "h1_gap", "h2_gap", "N1_height_ratio", "N1_fill_factor", "N1_coil_diameter",
    "N1_coil_zgap", "N2_height_ratio", "N2_fill_factor", "N2_coil_diameter",
    "N2_coil_zgap", "N1_space_w", "N1_space_l", "N2_space_w", "N2_space_l",
    "N1_layer_gap", "N2_layer_gap", "N1_offset_ratio", "N2_offset_ratio",
    "N1_offset", "N2_offset", "cold_plate_x", "cold_plate_y", "cold_plate_z1",
    "cold_plate_z2", "mold_thick", "thermal_conductivity"
]

# Input values
values = [6.0,6.0,1.0,1.0,3000.0,284.0,23.1,67.08827428571428,5.0,111.2,20.016,5.893599999999999,0.64,0.94,9.556845714285714,0.7116800000000012,0.8,0.75,9.531428571428572,3.706666666666668,20.0,20.0,7.3,3.0,5.2,9.9,0.0,-0.47,0.0,-5.226399999999998,0.0,0.8,11.0,0.0,20.0,0.4]

# Convert to dictionary with np.float64 values
input_parameters = {k: np.float64(v) for k, v in zip(keys, values)}

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()
simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell()

# 9. 결과 리포팅
simulation_runner.get_simulation_results(input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()




In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
simulation_runner.results_df

In [ ]:
simulation_runner.input_df

In [ ]:
import run_simulation

# 1. Simulation 클래스의 인스턴스를 생성합니다.
simulation_runner = run_simulation.Simulation()

# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
# 이 과정에서 내부에 self.design 객체가 생성됩니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
# 이 메서드는 초기화된 design 객체를 사용하게 됩니다.
#
#param = [10,10,1,1,2331,297.1,15,88.24156818,23.8,65.5,6.8775,6.533625,0.5,0.74,2.203181818,0.8515,0.65,0.63,2.438386364,1.575275,27.5,9.9,5.1,18.6,3.1,4.8,0.58,0.43,9.4975,4.928875,2.4,13.2,21.4,0.32]
input_parameters = simulation_runner.create_input_parameter(param_list=None)

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()
simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell()

# 9. 결과 리포팅
simulation_runner.get_simulation_results(input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()

# 15. 결과 저장
simulation_runner.save_results_to_csv(simulation_runner.results_df)




In [ ]:
parameters = [
    [simulation_runner.maxwell_design.winding1, f"P_winding1", "EMLoss"],
    [simulation_runner.maxwell_design.winding2, f"P_winding2", "EMLoss"],
    [simulation_runner.maxwell_design.core, f"P_Core", "CoreLoss"],
    [simulation_runner.maxwell_design.leg_left , f"B_mean_leg_left", "B_mean"],
    [simulation_runner.maxwell_design.leg_right, f"B_mean_leg_right", "B_mean"],
    [simulation_runner.maxwell_design.leg_top, f"B_mean_leg_top", "B_mean"],
    [simulation_runner.maxwell_design.leg_bottom, f"B_mean_leg_bottom", "B_mean"],
]

name_list = []
result_expressions = []
report = None

result_expressions, name_list = simulation_runner.icepak_design._add_calculator_expression(parameters=parameters)

In [ ]:
def get_calculator_parameter(self, dir=None, parameters=[], mod="write", import_report=None) :
        """
        example :
        parameters2 = []
        parameters2.append([winding1, "P_LV", "EMLoss"])
        # ... (rest of example)
        """
        name_list = []
        result_expressions = []
        report = None

        if mod == "write" :
            result_expressions, name_list = self._add_calculator_expression(parameters=parameters)
            report = self._create_report(report_name = "calculator_report", result_expressions = result_expressions, category = "Fields")
            if not hasattr(self, "report_list"):
                self.report_list = {}
            self.report_list["icepak_calc_report"] = report
        elif mod == "read" :
            report = self.report_list.get("icepak_calc_report", import_report)
            # In "read" mode, reconstruct the expression and name lists for column mapping
            for obj, name, expression_type in parameters:
                name_list.append(name)
                obj_name = obj.name if hasattr(obj, 'name') else obj
                
                if expression_type == "Temp_max":
                    expr_name = f"Temp_max_{obj_name}"
                elif expression_type == "Temp_mean":
                    expr_name = f"Temp_mean_{obj_name}"
                elif expression_type == "B_mean":
                    expr_name = f"B_mean_{obj_name}"
                else: 
                    expr_name = f"P_{obj_name}"
                result_expressions.append(expr_name)
        
        if not report:
             return None, pd.DataFrame(columns=[p[1] for p in parameters])

        export_data = self.post.export_report_to_csv(project_dir=dir, plot_name=report.plot_name)
        data = pd.read_csv(export_data)

        # Create a mapping from the actual column names in the CSV to the desired names
        rename_mapping = {}
        for expr, desired_name in zip(result_expressions, name_list):
            for col in data.columns:
                if expr in col:
                    rename_mapping[col] = desired_name
                    break
        
        # Rename columns and select only the ones we need
        output_df = data.rename(columns=rename_mapping)

        # Ensure all desired columns exist, adding missing ones with NaN
        for name in name_list:
            if name not in output_df.columns:
                output_df[name] = np.nan

        output_df = output_df[name_list]
        output_df.dropna(inplace=True)

        output_df.to_csv("icepak_calculator.csv")
        
        return report, output_df

In [ ]:
import os

from report import (
    get_input_parameter, get_maxwell_magnetic_parameter,
    get_maxwell_calculator_parameter, get_convergence_report, get_icepak_calculator_parameter
)



current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "simulation", f"{simulation_runner.PROJECT_NAME}")
        
        # Get results using the calculator
# icepak_results_df = get_icepak_calculator_parameter(simulation_runner.icepak_design, dir=folder_path)  

simulation_runner.icepak_design.winding1

In [ ]:

params = [
    [design.core, f"Temp_max_core", "Temp_max"],
    [design.core, f"Temp_mean_core", "Temp_mean"],
    [design.winding1, f"Temp_max_winding1", "Temp_max"],
    [design.winding1, f"Temp_mean_winding1", "Temp_mean"],
    [design.winding2, f"Temp_max_winding2", "Temp_max"],
    [design.winding2, f"Temp_mean_winding2", "Temp_mean"],
    [design.leg_left, f"Temp_max_leg_left", "Temp_max"],
    [design.leg_right, f"Temp_max_leg_right", "Temp_max"],
    [design.leg_top, f"Temp_max_leg_top", "Temp_max"],
    [design.leg_bottom, f"Temp_max_leg_bottom", "Temp_max"],
    [design.leg_left, f"Temp_mean_leg_left", "Temp_mean"],
    [design.leg_right, f"Temp_mean_leg_right", "Temp_mean"],
    [design.leg_top, f"Temp_mean_leg_top", "Temp_mean"],
    [design.leg_bottom, f"Temp_mean_leg_bottom", "Temp_mean"],
]

df = design.get_calculator_parameter(dir=dir, parameters=params)





In [ ]:
simulation_runner.w1

In [ ]:
simulation_runner.winding1.faces

In [ ]:
input_parameter = sim1.create_input_parameter()

sim1.set_variable(input_parameter)


In [ ]:
sim1.create_core()


In [ ]:
def offset_calculation(coil_diameter, h1, height_ratio, offset_z) :

    plus = coil_diameter/2
    minus = ((h1*height_ratio) - coil_diameter/2)

    offset = (plus + minus)/2 - plus + offset_z
    offset = (plus + minus)/2 - plus

    return offset

offset1 = offset_calculation(sim1.N1_coil_diameter, sim1.h1, sim1.N1_height_ratio, sim1.N1_offset)
winding_params1 = {
    "N" : sim1.N1,
    "N_layer" : sim1.N1_layer,
    "x" : "w1 + 2*N1_space_w",
    "y" : "l1 + 2*N1_space_l",
    "coil_diameter" : "N1_coil_diameter",
    "coil_zgap" : "N1_coil_zgap",
    "coil_layer_x_gap" : "N1_layer_gap",
    "coil_layer_y_gap" : "N1_layer_gap",
    "color" : [255, 50, 50],
    "transparency" : 0,
    "offset" : ["0mm", "-(l1+l2)/2", f"{offset1}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding1", **winding_params1)

offset2 = offset_calculation(sim1.N2_coil_diameter, sim1.h1, sim1.N2_height_ratio, sim1.N2_offset)
winding_params2 = {
    "N" : sim1.N2,
    "N_layer" : sim1.N2_layer,
    "x" : "w1 + 2*N2_space_w",
    "y" : "l1 + 2*N2_space_l",
    "coil_diameter" : "N2_coil_diameter",
    "coil_zgap" : "N2_coil_zgap",
    "coil_layer_x_gap" : "N2_layer_gap",
    "coil_layer_y_gap" : "N2_layer_gap",
    "color" : [50, 50, 255],
    "transparency" : 0,
    "offset" : ["0mm", "(l1+l2)/2", f"{offset2}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding2", **winding_params2)

In [ ]:
import subprocess
import time
import logging
import os
import platform

logging.basicConfig(filename='run_debug.log', level=logging.DEBUG)

script_name = "run_simulation.py"
script_path = os.path.join(os.getcwd(), script_name)

os_name = platform.system()
if os_name == "Windows":
    num_processes = 10 # number of subprocess
else :
    num_processes = 36 # number of subprocess


processes = []

log_dir = './simul_log'
os.makedirs(log_dir, exist_ok=True)

for i in range(num_processes):

    file_path = "simulog_num.txt"

    # 파일이 존재하지 않으면 생성
    if not os.path.exists(file_path):
        with open(file_path, "w", encoding="utf-8") as file:
            file.write("1")

    # 읽기/쓰기 모드로 파일 열기
    with open(file_path, "r+", encoding="utf-8") as file:

            # 파일에서 값 읽기
            content = int(file.read().strip())
            content += 1

            # 파일 포인터를 처음으로 되돌리고, 파일 내용 초기화 후 새 값 쓰기
            file.seek(0)
            file.truncate()
            file.write(str(content))

    log_file = open(f'./simul_log/process_{content}.log', 'w')
    p = subprocess.Popen(
        f'python {script_path}',
        shell=True,
        stdout=log_file,
        stderr=subprocess.STDOUT
    )
    processes.append((p, log_file))
    time.sleep(30)

for idx, (p, log_file) in enumerate(processes):
    p.wait()
    log_file.write(f"\nProcess {idx} finished with return code {p.returncode}\n")
    log_file.close()
